In [ ]:
import numpy as np
import json
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from skimage.draw import disk
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Activation, BatchNormalization, Input, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
# Maximum number of cores we expect in any image
MAX_CORES = 256  # Adjust this number based on your dataset

def create_label_array(json_data):
    labels = []
    for item in json_data:
        labels.append([item['x'], item['y'], item['radius']])
    # If there are fewer cores than MAX_CORES, we pad the remaining values with -1
    while len(labels) < MAX_CORES:
        labels.append([-1, -1, -1])  # Padding
    labels_flat = np.array(labels).flatten()
    # Normalize and flatten the labels to be between 0 and 1, except for padding values which remain -1
    for i in range(0, len(labels_flat), 3):
        if labels_flat[i] != -1:
            labels_flat[i] /= 1024
            labels_flat[i+1] /= 1024
            labels_flat[i+2] /= 1024
    return labels_flat

def load_images_and_labels(image_dir, label_dir):
    image_files = [os.path.join(image_dir, file) for file in sorted(os.listdir(image_dir)) if file.endswith('.png')]
    label_files = [os.path.join(label_dir, file) for file in sorted(os.listdir(label_dir)) if file.endswith('.json')]

    images = []
    labels = []

    for image_file, label_file in zip(image_files, label_files):
        # Load image
        image = img_to_array(load_img(image_file, color_mode='rgb'))
        images.append(image / 255.0)  # Normalize the image

        # Load corresponding label
        with open(label_file, 'r') as file:
            json_data = json.load(file)
        label = create_label_array(json_data)
        labels.append(label)

    return np.array(images), np.array(labels)


# Usage
image_dir = './TMA_WSI_Padded_PNGs'
label_dir = './TMA_WSI_Labels_updated'
images, labels = load_images_and_labels(image_dir, label_dir)


In [ ]:

def conv_block(input_tensor, num_filters, kernel_size=3, do_batch_norm=True):
    # A conv block consists of two convolutions, each followed by a batch normalization and a relu activation.
    x = Conv2D(num_filters, kernel_size, padding='same', kernel_initializer='he_normal')(input_tensor)
    if do_batch_norm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(num_filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    if do_batch_norm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def unet_regression(input_size=(1024, 1024, 3), num_filters=64, depth=4, dropout=0.5, batch_norm=True, max_cores = 256):
    # INPUT LAYER
    inputs = Input(input_size)
    # CONTRACTING PATH
    conv_blocks = []
    x = inputs
    for i in range(depth):
        x = conv_block(x, num_filters * (2**i), do_batch_norm=batch_norm)
        conv_blocks.append(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        if dropout:
            x = Dropout(dropout)(x)

    # BOTTLENECK
    x = conv_block(x, num_filters * (2**(depth)), do_batch_norm=batch_norm)
    
    # EXPANSIVE PATH
    for i in reversed(range(depth)):
        num_filters_exp = num_filters * (2**i)
        x = UpSampling2D(size=(2, 2))(x)
        x = concatenate([x, conv_blocks[i]], axis=3)
        x = conv_block(x, num_filters_exp, do_batch_norm=batch_norm)


    # OLD OUTPUT LAYER
    # output = Conv2D(1, 1, activation='sigmoid')(x)
    # model = Model(inputs=inputs, outputs=output)

    # NEW OUTPUT LAYER
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout)(x)
    # Adjust the number of outputs to be 3 * MAX_CORES
    outputs = Dense(3 * max_cores, activation='linear')(x)  # 3 values for each core: x, y, and radius

    model = Model(inputs=inputs, outputs=outputs)

    return model


model = unet_regression(input_size=(1024, 1024, 3), num_filters=64, depth=4, dropout=0.5, batch_norm=True, max_cores = 256)
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import MeanSquaredError

# Assuming 'images' and 'labels' are loaded and preprocessed correctly

# Define a custom loss function that can handle the padding
def custom_loss(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, -1), tf.float32)  # Create a mask for the padding
    loss = MeanSquaredError()(y_true * mask, y_pred * mask)  # Apply the mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)  # Normalize by the number of non-padded entries

# Create the U-Net model (ensure this function is defined correctly)
model = unet_regression(input_size=(1024, 1024, 3), num_filters=32, depth=4, dropout=0.5, batch_norm=True)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss=custom_loss)

# Set up callbacks for learning rate scheduling and checkpointing
checkpoint = ModelCheckpoint('model_checkpoint.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min', min_lr=1e-6)

# Split your data into training and validation sets
# This is a simple split, consider using sklearn's train_test_split for a random split
# val_split = 0.1  # Use 10% of data for validation
# num_val_samples = int(val_split * len(images))
# train_images, val_images = images[:-num_val_samples], images[-num_val_samples:]
# train_labels, val_labels = labels[:-num_val_samples], labels[-num_val_samples:]



In [ ]:

# # Train the model
# history = model.fit(
#     train_images, 
#     train_labels, 
#     validation_data=(val_images, val_labels),
#     epochs=5,  # Set the number of epochs
#     batch_size=3,  # Set the batch size
#     callbacks=[checkpoint, reduce_lr]
# )

# Train the model
history = model.fit(
    images, 
    labels, 
    epochs=5,  # Set the number of epochs
    batch_size=3,  # Set the batch size
    callbacks=[checkpoint, reduce_lr]
)